# 3. 다음에 볼 영화 예측하기 [프로젝트]
# 2024.9.3
# 구태훈

Movielens 1M Dataset을 기반으로, Session based Recommendation 시스템을 제작

여기서 이전 실습 내역과 가장 크게 다른 부분은 바로 SessionID 대신 UserID 항목이 들어갔다는 점입니다. 이 데이터셋은 명확한 1회 세션의 SessionID를 포함하지 않고 있습니다. 그래서 이번에는 UserID가 SessionID 역할을 해야 합니다.

Rating 정보가 포함되어 있습니다. 이전 실습 내역에서는 이런 항목이 포함되어 있지 않았으므로, 무시하고 제외할 수 있습니다. 하지만, 직전에 봤던 영화가 맘에 들었는지가 비슷한 영화를 더 고르게 하는 것과 상관이 있을 수도 있습니다. 아울러, Rating이 낮은 데이터를 어떻게 처리할지도 고민해야 합니다.

Time 항목에는 UTC time 가 포함되어, 1970년 1월 1일부터 경과된 초 단위 시간이 기재되어 있습니다.

위와 같은 정보를 바탕으로 오늘의 실습과정과 유사한 프로젝트 과정을 진행해 보겠습니다.

Step 1. 데이터의 전처리
위와 같이 간단히 구성해 본 데이터셋을 꼼꼼히 살펴보면서 항목별 기본 분석, session length, session time, cleaning 등의 작업을 진행합니다.
특히, 이 데이터셋에서는 Session이 아닌 UserID 단위로 데이터가 생성되어 있으므로, 이를 Session 단위로 어떻게 해석할지에 주의합니다.

Step 2. 미니 배치의 구성
실습 코드 내역을 참고하여 데이터셋과 미니 배치를 구성해 봅시다. Session-Parallel Mini-Batch의 개념에 따라, 학습 속도의 저하가 최소화될 수 있도록 구성합니다.
단, 위 Step 1에서 Session 단위를 어떻게 정의했느냐에 따라서 Session-Parallel Mini-Batch이 굳이 필요하지 않을 수도 있습니다.

Step 3. 모델 구성
이 부분도 실습 코드 내역을 참고하여 다양하게 모델 구조를 시도해 볼 수 있습니다.

Step 4. 모델 학습
다양한 하이퍼파라미터를 변경해 보며 검증해 보도록 합니다. 실습 코드에 언급되었던 Recall, MRR 등의 개념들도 함께 관리될 수 있도록 합니다.

Step 5. 모델 테스트
미리 구성한 테스트셋을 바탕으로 Recall, MRR 을 확인해 봅니다.

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Dropout, GRU, Embedding, TimeDistributed
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import legacy as legacy_optimizers
from pathlib import Path
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"tensorflow version: {tf.__version__}")


# Step 1. 데이터 전처리

# 데이터 로드 및 전처리
data_path = Path('.')
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3],
                       dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows, engine='python')
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)

# 데이터를 시간순으로 정렬합니다.
data.sort_values(['UserId', 'Time'], inplace=True)

# 1시간 내의 연속된 평가를 하나의 세션으로 간주하고 정의합니다.
data['Time'] = pd.to_datetime(data['Time'], unit='s')
data['TimeDiff'] = data.groupby('UserId')['Time'].diff()
session_threshold = pd.Timedelta('1 hours')

# 평점이 3점 미만인 데이터와 길이가 2 미만인 세션을 제거합니다.
data['SessionId'] = (data['TimeDiff'] > session_threshold).astype('int').cumsum()
data = data[data['Rating'] >= 3]
session_length = data.groupby('SessionId').size()
short_session_ids = session_length[session_length < 2].index
data = data[~data['SessionId'].isin(short_session_ids)]

print(f"전처리 후 데이터 샘플:\n{data.head()}\n")
print(f"총 행 수: {len(data)}")

# 각 아이템에 고유한 인덱스를 부여합니다.
item_ids = data['ItemId'].unique()
item_id_map = {old_id: new_id + 1 for new_id, old_id in enumerate(item_ids)}  # 0은 패딩용
data['ItemIdx'] = data['ItemId'].map(item_id_map)

num_items = len(item_id_map) + 1  # 패딩을 위해 1 추가
print("Number of unique items (including padding):", num_items)

args = {
    'batch_size': 32,
    'hidden_size': 256,  # 증가된 히든 사이즈
    'dropout_rate': 0.3,  # 드롭아웃 비율 조정
    'learning_rate': 0.0005,  # 낮춘 학습률
    'epochs': 10,  # 학습 에폭 수 증가
    'k': 20,
    'num_items': num_items
}


# Step 2. 미니 배치 구성

# 효율적으로 관리하기 위한 클래스입니다.
class GRURecommender(tf.keras.Model):
    def __init__(self, num_items, hidden_size, dropout_rate):
        super(GRURecommender, self).__init__()
        self.item_embedding = Embedding(num_items, hidden_size, mask_zero=True)
        self.gru_layer1 = GRU(hidden_size, return_sequences=True)
        self.gru_layer2 = GRU(hidden_size, return_sequences=True)
        self.gru_layer3 = GRU(hidden_size, return_sequences=True)
        self.attention = Attention()  # Attention 메커니즘 추가
        self.layer_norm = LayerNormalization()  # 정규화 레이어 추가
        self.dropout = Dropout(dropout_rate)
        self.output_layer = TimeDistributed(Dense(num_items))

    def call(self, inputs, training=False):
        x = self.item_embedding(inputs)
        mask = tf.not_equal(inputs, 0)
        x = self.gru_layer1(x, mask=mask)
        x = self.gru_layer2(x, mask=mask)
        x = self.gru_layer3(x, mask=mask)
        # Attention을 GRU의 두 번째 레이어와 결합
        attn_output = self.attention([x, x], training=training)
        x = Add()([x, attn_output])  # GRU 출력과 Attention을 결합
        x = self.layer_norm(x)  # 정규화
        x = self.dropout(x, training=training)
        return self.output_layer(x)
        
# 데이터를 배치 단위로 제공합니다.
class SessionDataLoader:
    def __init__(self, dataset, batch_size=50, max_len=50):
        self.dataset = dataset
        self.batch_size = batch_size
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset.session_idx) // self.batch_size

    def __iter__(self):
        session_idx = np.random.permutation(self.dataset.session_idx)
        offsets = self.dataset.click_offsets
        itemidx = self.dataset.itemidx

        for start_idx in range(0, len(session_idx), self.batch_size):
            batch_idx = session_idx[start_idx:start_idx + self.batch_size]
            batch_start = offsets[batch_idx]
            batch_end = offsets[batch_idx + 1]
            
            max_len = min(self.max_len, max(end - start for start, end in zip(batch_start, batch_end)))
            
            batch_inputs = np.zeros((len(batch_idx), max_len), dtype=np.int32)
            batch_targets = np.zeros((len(batch_idx), max_len), dtype=np.int32)
            mask = np.zeros((len(batch_idx), max_len), dtype=np.bool_)

            for i, (start, end) in enumerate(zip(batch_start, batch_end)):
                session_len = min(self.max_len, end - start)
                session_items = itemidx[start:start+session_len]
                batch_inputs[i, :session_len-1] = session_items[:-1]
                batch_targets[i, :session_len-1] = session_items[1:]
                mask[i, :session_len-1] = True

            yield (tf.convert_to_tensor(batch_inputs, dtype=tf.int32),
                   tf.convert_to_tensor(batch_targets, dtype=tf.int32),
                   tf.convert_to_tensor(mask, dtype=tf.bool))


# Step 3. 모델 구성

# 순환 신경망 기반의 추천 모델을 정의합니다.
# 임베딩층, GRU층, 드롭아웃층, 출력층으로 구성됩니다.
# 이 구조는 시퀀스 데이터(사용자의 아이템 선택 history)를 효과적으로 학습할 수 있습니다.

class GRURecommender(tf.keras.Model):
    def __init__(self, num_items, hidden_size, dropout_rate):
        super(GRURecommender, self).__init__()
        self.item_embedding = Embedding(num_items, hidden_size, mask_zero=True)
        self.gru_layer = GRU(hidden_size, return_sequences=True)
        self.dropout = Dropout(dropout_rate)
        self.output_layer = TimeDistributed(Dense(num_items))

    def call(self, inputs, training=False):
        x = self.item_embedding(inputs)
        mask = tf.not_equal(inputs, 0)
        x = self.gru_layer(x, mask=mask)
        x = self.dropout(x, training=training)
        return self.output_layer(x)


# Step 4. 모델 학습

# 이 함수는 한 배치의 데이터로 모델을 학습시키는 과정을 정의합니다.

@tf.function
def train_step(model, optimizer, inputs, targets, mask):
    with tf.GradientTape() as tape:
        logits = model(inputs, training=True)
        targets = tf.cast(targets, tf.int64)
        mask = tf.cast(mask, tf.float32)
        loss = tf.keras.losses.sparse_categorical_crossentropy(targets, logits, from_logits=True)
        masked_loss = loss * mask
        loss = tf.reduce_sum(masked_loss) / tf.reduce_sum(mask)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


# 이 함수는 전체 데이터셋에 대해 여러 epoch 동안 학습을 진행합니다. 
# 학습 중 손실이 감소하는 것을 확인할 수 있습니다.

def train(model, optimizer, train_loader, epochs):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        total_loss = 0
        num_batches = 0
        for inputs, targets, mask in train_loader:
            loss = train_step(model, optimizer, inputs, targets, mask)
            total_loss += loss
            num_batches += 1
            if num_batches % 100 == 0:
                print(f"Batch {num_batches}, Loss: {loss:.4f}")
        avg_loss = total_loss / num_batches
        print(f"Average Loss: {avg_loss:.4f}")


# Step 5. 모델 테스트

# 이 함수는 학습된 모델의 성능을 평가합니다.

def evaluate(model, test_loader, k):
    hit_sum, mrr_sum, num_samples = 0, 0, 0
    for inputs, targets, mask in test_loader:
        logits = model(inputs, training=False)
        last_item_logits = logits[:, -1, :]
        _, top_k = tf.nn.top_k(last_item_logits, k=k)
        
        for i, (pred, target, m) in enumerate(zip(top_k.numpy(), targets.numpy(), mask.numpy())):
            true_targets = target[m]
            if len(true_targets) > 0:
                target = true_targets[-1]  # 마지막 아이템
                if target in pred:
                    hit_sum += 1
                    mrr_sum += 1.0 / (np.where(pred == target)[0][0] + 1)
                num_samples += 1

    # Hit Rate : 모델이 추천한 상위 k개 아이템 중 실제 선택된 아이템이 포함된 비율을 계산합니다. 높을수록 상위 k개 안에 추천할 확률이 높습니다.  
    hit_rate = hit_sum / num_samples if num_samples > 0 else 0

    # MRR(Mean Reciprocal Rank) : 실제 선택된 아이템의 순위의 역수의 평균을 계산합니다. 높을수록 모델이 실제로 상위 순위에 추천합니다.
    mrr = mrr_sum / num_samples if num_samples > 0 else 0
    return hit_rate, mrr

# GPU 메모리 증가 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 데이터 준비
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

train_dataset = SessionDataset(train_data)
test_dataset = SessionDataset(test_data)

# 기존 학습 및 평가 함수는 유지하고 하이퍼파라미터 및 개선된 모델로 실행합니다.
model = GRURecommender(num_items=args['num_items'], hidden_size=args['hidden_size'], dropout_rate=args['dropout_rate'])
optimizer = legacy_optimizers.Adam(learning_rate=args['learning_rate'])

# 학습 시작
train_loader = SessionDataLoader(train_dataset, batch_size=args['batch_size'], max_len=50)
test_loader = SessionDataLoader(test_dataset, batch_size=args['batch_size'], max_len=50)

# 더미 입력으로 모델 테스트
dummy_inputs = tf.zeros((args['batch_size'], 50), dtype=tf.int32)
_ = model(dummy_inputs)

print("Starting training with improved model...")
train(model, optimizer, train_loader, args['epochs'])

print("Training completed. Starting evaluation...")
hit_rate, mrr = evaluate(model, test_loader, args['k'])
print(f"Hit Rate @ {args['k']}: {hit_rate:.4f}")
print(f"MRR @ {args['k']}: {mrr:.4f}")

pandas version: 2.2.2
numpy version: 1.26.4
tensorflow version: 2.15.0
전처리 후 데이터 샘플:
    UserId  ItemId  Rating                Time        TimeDiff  SessionId
31       1    3186       4 2000-12-31 22:00:19             NaT          0
22       1    1270       5 2000-12-31 22:00:55 0 days 00:00:36          0
27       1    1721       4 2000-12-31 22:00:55 0 days 00:00:00          0
37       1    1022       5 2000-12-31 22:00:55 0 days 00:00:00          0
24       1    2340       3 2000-12-31 22:01:43 0 days 00:00:48          0

총 행 수: 830594
Number of unique items (including padding): 3623
Starting training with improved model...
Epoch 1/10
Batch 100, Loss: 7.3196
Batch 200, Loss: 7.2884
Average Loss: 7.4653
Epoch 2/10
Batch 100, Loss: 7.1698
Batch 200, Loss: 7.4170
Average Loss: 7.1060
Epoch 3/10
Batch 100, Loss: 6.8551
Batch 200, Loss: 6.9837
Average Loss: 6.8989
Epoch 4/10
Batch 100, Loss: 6.7312
Batch 200, Loss: 6.5883
Average Loss: 6.5985
Epoch 5/10
Batch 100, Loss: 6.2956
Batch 200, 

# 회고
- 프로젝트에서 제시하는 5단계의 코드 작성과정을 이해하는데 어려움이 있었습니다. 데이터 로드부터 전처리 일부까지 작성하고 이후는 claude와 chatgpt의 도움을 받았습니다.
- 5단계의 틀은 첫날부터 잡혔지만 shape과 dtype이 올바르게 전달되지 않아서 3일간 claude의 유료 서비스 양이 소진될 때까지 계속 수정을 거듭했습니다. 수정을 하면서 배우는 독특한 경험을 했습니다.
- 2일째부터 코랩 환경에 회의를 느껴 로컬에서 설치를 시작했습니다. 맥미니 M1에 메탈 gpu 설정을 하면 cuda만큼은 아니라도 cpu 보다 6배 정도 빠르다고 해서 설정을 이틀만에 마쳤습니다. 주피터 노트북을 터미널에서 conda activate tf-metal 모드로 활성화해서 실행을 하니 투명하게 작업환경이 보이는 것 같습니다.
- Loss가 생각보다 안내려가서 epoch 를 줄여서 테스트하는 것으로 만족했습니다. 예측 오류가 큰 상태입니다.
- Hit Rate는 20개의 영화를 추천한 중에 5.3%가 사용자가 선택한 영화에 포함됩니다. 
- MRR은 사용자가 선택한 영화가 추천 리스트의 상위권에 거의 나오지 않습니다.
- 성능이 낮은 이유는 데이터의 품질, 하이퍼파라미터의 조절필요 등이 있겠습니다.
- 다음과 같은 간단한 하이퍼파라미터의 조절로 평가 수치가 상승했습니다.
- 'batch_size': 16 -> 32, 'hidden_size': 32 -> 256, 
- Average Loss: 6.8120 -> 5.7897
- Hit Rate @ 20: 0.0530 -> 0.0679
- MRR @ 20: 0.0128 -> 0.0136